In [1]:
import torch
from torch import nn # nn contains all of pytorch's building blocks to a neural network
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch.utils.data as dataUtil

/home/operation/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.concat([
                # pd.read_csv("./VariablesToSave.1.csv",sep=";"),
                # pd.read_csv("./VariablesToSave.2.csv",sep=";"),
                # pd.read_csv("./VariablesToSave.3.csv",sep=";"),
                pd.read_csv("./VariablesToSave.4.csv",sep=";")
                ])

In [3]:
data = data.drop("Name",axis="columns")
data = data.drop("lrSetPointTension",axis="columns")
data = data.drop("lrDeadTimeCompensation",axis="columns")
data = data.drop("lrTotalTorqueAdditive",axis="columns")
data = data.drop("lrAccelerationRead",axis="columns")
data = data.drop("lrVelocityRead",axis="columns")
data = data.drop("lrActDiameter",axis="columns")
data = data.drop("lrSyncVelocity",axis="columns")
# data = data.drop("aMeasuredTension[1]",axis="columns")
# data = data.drop("aMeasuredTension[2]",axis="columns")

# data.to_csv("./data.csv",encoding='utf-8', index=False,sep=";")

In [4]:
y=torch.tensor(data["lrMeasuredForce"],dtype=torch.float32).unsqueeze(dim=1)
data = data.drop("lrMeasuredForce",axis="columns")
X=torch.tensor(list(data.values),dtype=torch.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X[0:10])

/tmp/ipykernel_2411744/2929382495.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:204.)
  X=torch.tensor(list(data.values),dtype=torch.float32)


tensor([[42.7680, 42.7680, -0.2985],
        [42.8706, 42.7680, -0.4056],
        [42.8166, 42.8706, -0.3468],
        [42.9192, 42.8166, -0.4540],
        [42.8706, 42.9192, -0.4009],
        [42.9192, 42.8706, -0.4510],
        [42.9192, 42.9192, -0.4496],
        [42.9678, 42.9192, -0.4998],
        [42.9678, 42.9678, -0.4986],
        [42.9678, 42.9678, -0.4974]])


In [5]:
# from sklearn.linear_model import Ridge
# reg = Ridge(alpha=1000).fit(X.detach().numpy(), y.detach().numpy())
from sklearn.svm import SVR
reg= SVR(kernel="rbf", C=100, gamma=0.1, epsilon=0.1).fit(X.detach().numpy(), y.detach().numpy()[:,0])
print(reg.score(X, y))
y_predicted=reg.predict(X)
plt.plot(y,label="real")
plt.plot(y_predicted,label="predicted")
plt.legend()
plt.show()

In [ ]:
# tension_1 tension_2 Pid velocity
start=40500
end=109000
history=[]
tension_1 =X[start][0]
tension_2 =X[start][1]
controllerOutput = X[start:end][:,2]
tension_real =  y[start:end]


for i in range(len(controllerOutput)):
    #create real tensor 
    # input = torch.tensor([tension_1,tension_2,controllerOutput[i],speed[i]],dtype=torch.float32)
    input = torch.tensor([tension_1,tension_2,controllerOutput[i]],dtype=torch.float32)
    tension_0 = reg.predict([input.detach().numpy()])[0][0]
    history.append(tension_0)
    tension_2=tension_1
    tension_1=tension_0
plt.plot(history,label="simulated")
plt.plot(tension_real,label="real")
# plt.plot(controllerOutput,label="outputs")
plt.legend()
plt.show()